# Stress analysis for each element 

In [23]:
name = 'yannis'

# Import modules 

In [24]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.panels import *
from formulas.helpers import *
from formulas.abd_matrix import *
from formulas.Strength import * 

# Import needed files 

## Import panel stresses

In [25]:
PlyStressesX = pd.read_csv(f'../data/{name}/compositePanelXX.csv')
PlyStressesX = PlyStressesX.drop(columns=['FileID', 'Step'])
PlyStressesY = pd.read_csv(f'../data/{name}/compositePanelYY.csv')
PlyStressesY = PlyStressesY.drop(columns=['FileID', 'Step'])
PlyStressesXY = pd.read_csv(f'../data/{name}/compositePanelXY.csv')
PlyStressesXY = PlyStressesXY.drop(columns=['FileID', 'Step'])
PlyStressesX

,Elements,Loadcase,Layer,Composite Stresses:Normal X Stress
0,1,1,Ply 1,-35.419704
1,2,1,Ply 1,25.747299
2,3,1,Ply 1,2.299717
3,4,1,Ply 1,-22.946745
4,5,1,Ply 1,14.230029
...,...,...,...,...
1435,26,3,Ply 9,15.286470
1436,27,3,Ply 9,-46.393162
1437,28,3,Ply 9,66.984344
1438,29,3,Ply 9,18.347198


In [26]:
PlyStressesPanel=pd.merge(PlyStressesX,PlyStressesY, on=['Elements', 'Layer', 'Loadcase'])
PlyStressesPanel = pd.merge(PlyStressesPanel, PlyStressesXY, on=['Elements', 'Layer', 'Loadcase'])
PlyStressesPanel = PlyStressesPanel.rename(columns={'Composite Stresses:Normal X Stress':'Normal_1', 'Composite Stresses:Normal Y Stress':'Normal_2', 'Composite Stresses:Shear XY Stress':'Shear_12' })
PlyStressesPanel

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12
0,1,1,Ply 1,-35.419704,-10.641421,11.146913
1,2,1,Ply 1,25.747299,-3.627014,4.989726
2,3,1,Ply 1,2.299717,-11.597987,12.994295
3,4,1,Ply 1,-22.946745,-10.609735,8.319627
4,5,1,Ply 1,14.230029,-4.767473,4.595075
...,...,...,...,...,...,...
1435,26,3,Ply 9,15.286470,-2.278381,-12.591271
1436,27,3,Ply 9,-46.393162,-2.525533,-14.392484
1437,28,3,Ply 9,66.984344,-1.739914,-14.648998
1438,29,3,Ply 9,18.347198,-3.127240,-11.663470


## Import stringer plies 

In [27]:
StringerStrains = pd.read_csv(f'../data/{name}/stringer_strain.csv')
StringerStrains = StringerStrains.drop(columns=['FileID', 'Step'])
StringerStrains = StringerStrains.rename(columns={'Element Strains (1D):CBAR/CBEAM Axial Strain':'strainX'})
StringerStrains

,Elements,Loadcase,strainX
0,40,1,-0.001215
1,41,1,-0.001502
2,42,1,-0.001270
3,46,1,-0.001138
4,47,1,-0.001586
5,48,1,-0.001434
6,52,1,-0.001434
7,53,1,-0.001586
8,54,1,-0.001138
9,58,1,-0.001270


# Constants for analysis 

In [28]:
# r = perpendicular 
# p = parallel
# c = compression 
# t = tension 
# Resistance values 

R_p_t = 3050
R_p_c = 1500
R_r_c = 50
R_r_t = 300 
R_rp = 100

# p values 
# Perpendicular, parallel
p_rp_c = 0.25
p_rp_t = 0.25

# Perpendicular, perpendicular
p_rr_c = 0.25
p_rr_t = 0.25

#Ply orientation in flange 
StringerFlange=[45,45,-45,-45,0,0,90,90,90,90,0,0,-45,-45,45,45]

# Personal data Provider 

In [29]:
personal_data = personal_data_provider(name)
E_11 = personal_data[0]
E_22 = personal_data[1]
G_12 = personal_data[2]
nu_12 = personal_data[3]

# Calculate RF and mode for panels

## Add the ultimate load factor 

In [ ]:
PlyStressesPanel['Normal_1'] = PlyStressesPanel['Normal_1'] * 1.5
PlyStressesPanel['Normal_2'] = PlyStressesPanel['Normal_2'] * 1.5
PlyStressesPanel['Shear_12'] = PlyStressesPanel['Shear_12'] * 1.5

In [30]:
PlyStressesPanel[['Mode', 'RF_IFF', 'RF_FF']] = PlyStressesPanel.apply(strength, R_p_t=R_p_t, R_p_c=R_p_c, R_r_c=R_r_c, R_r_t=R_r_t, 
                                                    R_rp=R_rp, p_rp_c=p_rp_c, p_rp_t=p_rp_t, p_rr_c=p_rr_c, p_rr_t=p_rr_t,
                                                    axis=1, result_type='expand')
PlyStressesPanel

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF
0,1,1,Ply 1,-35.419704,-10.641421,11.146913,C,4.501067,42.349309
1,2,1,Ply 1,25.747299,-3.627014,4.989726,C,12.815285,118.459027
2,3,1,Ply 1,2.299717,-11.597987,12.994295,C,4.104977,1326.250035
3,4,1,Ply 1,-22.946745,-10.609735,8.319627,C,4.599525,65.368749
4,5,1,Ply 1,14.230029,-4.767473,4.595075,C,10.111981,214.335472
...,...,...,...,...,...,...,...,...,...
1435,26,3,Ply 9,15.286470,-2.278381,-12.591271,B,8.309407,199.522841
1436,27,3,Ply 9,-46.393162,-2.525533,-14.392484,B,7.259558,32.332351
1437,28,3,Ply 9,66.984344,-1.739914,-14.648998,B,7.032113,45.533028
1438,29,3,Ply 9,18.347198,-3.127240,-11.663470,C,10.272719,166.237914


## Calculate RF_strength 

In [31]:
PlyStressesPanel['RF_strength'] = PlyStressesPanel[['RF_IFF', 'RF_FF']].min(axis=1)
PlyStressesPanel.head(5)

,Elements,Loadcase,Layer,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF,RF_strength
0,1,1,Ply 1,-35.419704,-10.641421,11.146913,C,4.501067,42.349309,4.501067
1,2,1,Ply 1,25.747299,-3.627014,4.989726,C,12.815285,118.459027,12.815285
2,3,1,Ply 1,2.299717,-11.597987,12.994295,C,4.104977,1326.250035,4.104977
3,4,1,Ply 1,-22.946745,-10.609735,8.319627,C,4.599525,65.368749,4.599525
4,5,1,Ply 1,14.230029,-4.767473,4.595075,C,10.111981,214.335472,10.111981


## Convert to output for panels 

In [ ]:
# We can drop the stresses 
PlyStressesPanel = PlyStressesPanel.drop(columns=['Normal_1', 'Normal_2', 'Shear_12'])

#Reorder plies 
PlyStressesPanel['Layer_num'] = PlyStressesPanel['Layer'].str.extract('(\d+)').astype(int)
PlyStressesPanel = PlyStressesPanel.sort_values(by='Layer_num')
PlyStressesPanel = PlyStressesPanel.drop(columns=['Layer'])
PlyStressesPanel = PlyStressesPanel.rename(columns={'Layer_num':'Layer'})

# Resort columns to desired order
PlyStressesPanel = PlyStressesPanel[['Elements', 'Layer', 'Loadcase', 'RF_FF', 'RF_IFF', 'Mode', 'RF_strength']]

# We need to analyse element 8 
PlyStressesPanel = PlyStressesPanel[PlyStressesPanel['Elements'] == 8]
#Now we separate the loadcases 
PlyStressesPanelLC1 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 1]
PlyStressesPanelLC2 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 2]
PlyStressesPanelLC3 = PlyStressesPanel[PlyStressesPanel['Loadcase'] == 3]
PlyStressesPanelLC3

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Yanni\AppData\Local\Temp\ipykernel_22616\823948087.py:5: SyntaxWarning: invalid escape sequence '\d'
  PlyStressesPanel['Layer_num'] = PlyStressesPanel['Layer'].str.extract('(\d+)').astype(int)


,Elements,Layer,Loadcase,RF_FF,RF_IFF,Mode,RF_strength
967,8,1,3,19.498084,5.117252,C,5.117252
1207,8,2,3,19.591953,5.144409,C,5.144409
1237,8,3,3,8.616170,26.487947,A,8.616170
1267,8,4,3,8.661670,26.273976,A,8.661670
1297,8,5,3,46.102979,7.802814,A,7.802814
1327,8,6,3,45.443687,7.836886,A,7.836886
1357,8,7,3,203.664456,8.280986,B,8.280986
1387,8,8,3,195.353729,8.329333,B,8.329333
1417,8,9,3,187.694665,8.378245,B,8.378245
997,8,10,3,180.613498,8.427730,B,8.427730


## Rename Loadcases 

In [33]:
# Loadcase 1
PlyStressesPanelLC1 = PlyStressesPanelLC1.rename(columns={'RF_FF':'R_FF_LC1', 'RF_IFF':'R_IFF_LC1', 'Mode':'Mode_LC1', 'RF_strength':'RF_strength_LC1'})
PlyStressesPanelLC1 = PlyStressesPanelLC1.reset_index()
PlyStressesPanelLC1 = PlyStressesPanelLC1.drop(columns=['index', 'Loadcase'])

# Loadcase 2
PlyStressesPanelLC2 = PlyStressesPanelLC2.rename(columns={'RF_FF':'R_FF_LC2', 'RF_IFF':'R_IFF_LC2', 'Mode':'Mode_LC2', 'RF_strength':'RF_strength_LC2'})
PlyStressesPanelLC2 = PlyStressesPanelLC2.reset_index()
PlyStressesPanelLC2 = PlyStressesPanelLC2.drop(columns=['index', 'Loadcase'])

#Loadcase 3 
PlyStressesPanelLC3 = PlyStressesPanelLC3.rename(columns={'RF_FF':'R_FF_LC3', 'RF_IFF':'R_IFF_LC3', 'Mode':'Mode_LC3', 'RF_strength':'RF_strength_LC3'})
PlyStressesPanelLC3 = PlyStressesPanelLC3.reset_index()
PlyStressesPanelLC3 = PlyStressesPanelLC3.drop(columns=['index', 'Loadcase'])
PlyStressesPanelLC1

,Elements,Layer,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1
0,8,1,193.452048,10.215215,C,10.215215
1,8,2,199.062609,10.146641,C,10.146641
2,8,3,22.593910,19.073085,B,19.073085
3,8,4,22.530385,18.720386,B,18.720386
4,8,5,15.388749,29.044169,A,15.388749
5,8,6,15.125258,29.040437,A,15.125258
6,8,7,65.127018,7.506532,C,7.506532
7,8,8,63.676906,7.391902,C,7.391902
8,8,9,62.289963,7.280676,C,7.280676
9,8,10,60.962154,7.172707,C,7.172707


## Combine Loadcases 

In [34]:
PlyStressesPanel = PlyStressesPanelLC1.merge(PlyStressesPanelLC2, on=['Elements', 'Layer'], suffixes=('_LC1', '_LC2'))
PlyStressesPanel = PlyStressesPanel.merge(PlyStressesPanelLC3, on=['Elements', 'Layer'])
PlyStressesPanel

,Elements,Layer,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1,R_FF_LC2,R_IFF_LC2,Mode_LC2,RF_strength_LC2,R_FF_LC3,R_IFF_LC3,Mode_LC3,RF_strength_LC3
0,8,1,193.452048,10.215215,C,10.215215,919.690432,8.387987,C,8.387987,19.498084,5.117252,C,5.117252
1,8,2,199.062609,10.146641,C,10.146641,1166.846723,8.308015,C,8.308015,19.591953,5.144409,C,5.144409
2,8,3,22.593910,19.073085,B,19.073085,18.973863,24.870732,C,18.973863,8.616170,26.487947,A,8.616170
3,8,4,22.530385,18.720386,B,18.720386,18.848230,24.152088,C,18.848230,8.661670,26.273976,A,8.661670
4,8,5,15.388749,29.044169,A,15.388749,17.334689,28.320937,C,17.334689,46.102979,7.802814,A,7.802814
5,8,6,15.125258,29.040437,A,15.125258,17.133671,27.815051,C,17.133671,45.443687,7.836886,A,7.836886
6,8,7,65.127018,7.506532,C,7.506532,469.241406,7.499531,C,7.499531,203.664456,8.280986,B,8.280986
7,8,8,63.676906,7.391902,C,7.391902,485.230375,7.408943,C,7.408943,195.353729,8.329333,B,8.329333
8,8,9,62.289963,7.280676,C,7.280676,502.347440,7.320492,C,7.320492,187.694665,8.378245,B,8.378245
9,8,10,60.962154,7.172707,C,7.172707,520.716271,7.234102,C,7.234102,180.613498,8.427730,B,8.427730


# Output panel 

In [35]:
PlyStressesPanel.to_excel(f'../data/{name}/output/panelStrength.xlsx')

# Calculate stringer strength 

In [36]:
StringerStrains.head(5)

,Elements,Loadcase,strainX
0,40,1,-0.001215
1,41,1,-0.001502
2,42,1,-0.001270
3,46,1,-0.001138
4,47,1,-0.001586


## Expand the dataframe 

In [37]:
# Create a DataFrame with PlyNumber 1-16
ply_numbers = pd.DataFrame({'PlyNumber': np.arange(1, 17),
                            'plyTheta': StringerFlange})

# Cross join to expand each element row to 16 plies
StringerStrains = StringerStrains.merge(ply_numbers, how='cross')
#StringerStrains = StringerStrains.drop(columns=['PlyNumber_x'])
StringerStrains.head(33)

,Elements,Loadcase,strainX,PlyNumber,plyTheta
0,40,1,-0.001215,1,45
1,40,1,-0.001215,2,45
2,40,1,-0.001215,3,-45
3,40,1,-0.001215,4,-45
4,40,1,-0.001215,5,0
5,40,1,-0.001215,6,0
6,40,1,-0.001215,7,90
7,40,1,-0.001215,8,90
8,40,1,-0.001215,9,90
9,40,1,-0.001215,10,90


In [38]:
StringerStrains[['Normal_1', 'Normal_2', 'Shear_12']] = StringerStrains.apply(
    calculateMatStress, EModulus1=E_11, EModulus2=E_22, ShearModulus=G_12, axis=1,
    result_type='expand'
)
StringerStrains

,Elements,Loadcase,strainX,PlyNumber,plyTheta,Normal_1,Normal_2,Shear_12
0,40,1,-0.001215,1,45,-74.950604,-7.478190,5.575598
1,40,1,-0.001215,2,45,-74.950604,-7.478190,5.575598
2,40,1,-0.001215,3,-45,-74.950604,-7.478190,-5.575598
3,40,1,-0.001215,4,-45,-74.950604,-7.478190,-5.575598
4,40,1,-0.001215,5,0,-146.190227,-3.710982,0.000000
...,...,...,...,...,...,...,...,...
571,60,3,-0.000346,12,0,-41.672474,-1.057839,0.000000
572,60,3,-0.000346,13,-45,-21.365157,-2.131707,-1.589361
573,60,3,-0.000346,14,-45,-21.365157,-2.131707,-1.589361
574,60,3,-0.000346,15,45,-21.365157,-2.131707,1.589361


## Now calculate the FF and the IFF 

### Add ultimate load factor 

In [ ]:
StringerStrains['Normal_1'] = StringerStrains['Normal_1'] * 1.5
StringerStrains['Normal_2'] = StringerStrains['Normal_2'] * 1.5
StringerStrains['Shear_12'] = StringerStrains['Shear_12'] * 1.5

In [39]:
StringerStrains[['Mode', 'RF_IFF', 'RF_FF']] = StringerStrains.apply(strength, R_p_t=R_p_t, R_p_c=R_p_c, R_r_c=R_r_c, R_r_t=R_r_t, 
                                                    R_rp=R_rp, p_rp_c=p_rp_c, p_rp_t=p_rp_t, p_rr_c=p_rr_c, p_rr_t=p_rr_t,
                                                    axis=1, result_type='expand')
StringerStrains

,Elements,Loadcase,strainX,PlyNumber,plyTheta,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF
0,40,1,-0.001215,1,45,-74.950604,-7.478190,5.575598,C,6.540674,20.013181
1,40,1,-0.001215,2,45,-74.950604,-7.478190,5.575598,C,6.540674,20.013181
2,40,1,-0.001215,3,-45,-74.950604,-7.478190,-5.575598,C,6.540674,20.013181
3,40,1,-0.001215,4,-45,-74.950604,-7.478190,-5.575598,C,6.540674,20.013181
4,40,1,-0.001215,5,0,-146.190227,-3.710982,0.000000,C,13.473524,10.260604
...,...,...,...,...,...,...,...,...,...,...,...
571,60,3,-0.000346,12,0,-41.672474,-1.057839,0.000000,C,47.266153,35.994983
572,60,3,-0.000346,13,-45,-21.365157,-2.131707,-1.589361,C,22.945183,70.207769
573,60,3,-0.000346,14,-45,-21.365157,-2.131707,-1.589361,C,22.945183,70.207769
574,60,3,-0.000346,15,45,-21.365157,-2.131707,1.589361,C,22.945183,70.207769


In [40]:
StringerStrains['RF_strength'] = StringerStrains[['RF_IFF', 'RF_FF']].min(axis=1)
StringerStrains.head(5)

,Elements,Loadcase,strainX,PlyNumber,plyTheta,Normal_1,Normal_2,Shear_12,Mode,RF_IFF,RF_FF,RF_strength
0,40,1,-0.001215,1,45,-74.950604,-7.478190,5.575598,C,6.540674,20.013181,6.540674
1,40,1,-0.001215,2,45,-74.950604,-7.478190,5.575598,C,6.540674,20.013181,6.540674
2,40,1,-0.001215,3,-45,-74.950604,-7.478190,-5.575598,C,6.540674,20.013181,6.540674
3,40,1,-0.001215,4,-45,-74.950604,-7.478190,-5.575598,C,6.540674,20.013181,6.540674
4,40,1,-0.001215,5,0,-146.190227,-3.710982,0.000000,C,13.473524,10.260604,10.260604


In [41]:
# We can drop the stresses 
StringerStrains = StringerStrains.drop(columns=['Normal_1', 'Normal_2', 'Shear_12','strainX', 'plyTheta'])

# Resort columns to desired order
StringerStrains = StringerStrains[['Elements', 'PlyNumber', 'Loadcase', 'RF_FF', 'RF_IFF', 'Mode', 'RF_strength']]

# We need to analyse element 8 
StringerStrains = StringerStrains[StringerStrains['Elements'] == 60]
#Now we separate the loadcases 
PlyStressesStringerLC1 = StringerStrains[StringerStrains['Loadcase'] == 1]
PlyStressesStringerLC2 = StringerStrains[StringerStrains['Loadcase'] == 2]
PlyStressesStringerLC3 = StringerStrains[StringerStrains['Loadcase'] == 3]
PlyStressesStringerLC1

,Elements,PlyNumber,Loadcase,RF_FF,RF_IFF,Mode,RF_strength
176,60,1,1,20.013186,6.540675,C,6.540675
177,60,2,1,20.013186,6.540675,C,6.540675
178,60,3,1,20.013186,6.540675,C,6.540675
179,60,4,1,20.013186,6.540675,C,6.540675
180,60,5,1,10.260607,13.473528,C,10.260607
181,60,6,1,10.260607,13.473528,C,10.260607
182,60,7,1,404.205839,4.446264,C,4.446264
183,60,8,1,404.205839,4.446264,C,4.446264
184,60,9,1,404.205839,4.446264,C,4.446264
185,60,10,1,404.205839,4.446264,C,4.446264


## Rename Loadcases 

In [42]:
# Loadcase 1
PlyStressesStringerLC1 = PlyStressesStringerLC1.rename(columns={'RF_FF':'R_FF_LC1', 'RF_IFF':'R_IFF_LC1', 'Mode':'Mode_LC1', 'RF_strength':'RF_strength_LC1'})
PlyStressesStringerLC1 = PlyStressesStringerLC1.reset_index()
PlyStressesStringerLC1 = PlyStressesStringerLC1.drop(columns=['index', 'Loadcase'])

# Loadcase 2
PlyStressesStringerLC2 = PlyStressesStringerLC2.rename(columns={'RF_FF':'R_FF_LC2', 'RF_IFF':'R_IFF_LC2', 'Mode':'Mode_LC2', 'RF_strength':'RF_strength_LC2'})
PlyStressesStringerLC2 = PlyStressesStringerLC2.reset_index()
PlyStressesStringerLC2 = PlyStressesStringerLC2.drop(columns=['index', 'Loadcase'])

#Loadcase 3 
PlyStressesStringerLC3 = PlyStressesStringerLC3.rename(columns={'RF_FF':'R_FF_LC3', 'RF_IFF':'R_IFF_LC3', 'Mode':'Mode_LC3', 'RF_strength':'RF_strength_LC3'})
PlyStressesStringerLC3 = PlyStressesStringerLC3.reset_index()
PlyStressesStringerLC3 = PlyStressesStringerLC3.drop(columns=['index', 'Loadcase'])
PlyStressesStringerLC1

,Elements,PlyNumber,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1
0,60,1,20.013186,6.540675,C,6.540675
1,60,2,20.013186,6.540675,C,6.540675
2,60,3,20.013186,6.540675,C,6.540675
3,60,4,20.013186,6.540675,C,6.540675
4,60,5,10.260607,13.473528,C,10.260607
5,60,6,10.260607,13.473528,C,10.260607
6,60,7,404.205839,4.446264,C,4.446264
7,60,8,404.205839,4.446264,C,4.446264
8,60,9,404.205839,4.446264,C,4.446264
9,60,10,404.205839,4.446264,C,4.446264


## Combine Loadcases 

In [43]:
PlyStressesStringer = PlyStressesStringerLC1.merge(PlyStressesStringerLC2, on=['Elements', 'PlyNumber'], suffixes=('_LC1', '_LC2'))
PlyStressesStringer = PlyStressesStringer.merge(PlyStressesStringerLC3, on=['Elements', 'PlyNumber'])
PlyStressesStringer

,Elements,PlyNumber,R_FF_LC1,R_IFF_LC1,Mode_LC1,RF_strength_LC1,R_FF_LC2,R_IFF_LC2,Mode_LC2,RF_strength_LC2,R_FF_LC3,R_IFF_LC3,Mode_LC3,RF_strength_LC3
0,60,1,20.013186,6.540675,C,6.540675,19.708971,6.441252,C,6.441252,70.207769,22.945183,C,22.945183
1,60,2,20.013186,6.540675,C,6.540675,19.708971,6.441252,C,6.441252,70.207769,22.945183,C,22.945183
2,60,3,20.013186,6.540675,C,6.540675,19.708971,6.441252,C,6.441252,70.207769,22.945183,C,22.945183
3,60,4,20.013186,6.540675,C,6.540675,19.708971,6.441252,C,6.441252,70.207769,22.945183,C,22.945183
4,60,5,10.260607,13.473528,C,10.260607,10.104638,13.268720,C,10.104638,35.994983,47.266153,C,35.994983
5,60,6,10.260607,13.473528,C,10.260607,10.104638,13.268720,C,10.104638,35.994983,47.266153,C,35.994983
6,60,7,404.205839,4.446264,C,4.446264,398.061605,4.378678,C,4.378678,1417.984601,15.597831,C,15.597831
7,60,8,404.205839,4.446264,C,4.446264,398.061605,4.378678,C,4.378678,1417.984601,15.597831,C,15.597831
8,60,9,404.205839,4.446264,C,4.446264,398.061605,4.378678,C,4.378678,1417.984601,15.597831,C,15.597831
9,60,10,404.205839,4.446264,C,4.446264,398.061605,4.378678,C,4.378678,1417.984601,15.597831,C,15.597831


## Output stringer

In [44]:
PlyStressesStringer.to_excel(f'../data/{name}/output/stringerStrength.xlsx')